**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [181]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [182]:
qnt_orig = 5
qnt_trans = 4
qnt_port = 2
qnt_cli = 1

In [183]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {
    0: 243138,
    1: 354063,
    2: 143703,
    3: 755250,
    4: 1253270,
}

demand = {O[0]: 2749424}

cap_transbordo = {
    K[0]: 691205,
    K[1]: 1709592,
    K[2]: 223253,
    K[3]: 125374
}

cap_port = {
    M[0]: 2577035,
    M[1]: 172389
}

In [184]:
cost = {
    (0, M[0]): 217.11, (0, M[1]): 276.58,
    (1, M[0]): 230.07, (1, M[1]): 261.53,
    (2, M[0]): 165.83, (2, M[1]): 226.65,
    (3, M[0]): 157.93, (3, M[1]): 184.61,
    (4, M[0]): 130.65, (4, M[1]): 186.55,
    
    (K[0], M[0]): 81.64, (K[0], M[1]): 153.16,
    (K[1], M[0]): 93.82, (K[1], M[1]): 141.19,
    (K[2], M[0]): 111.78, (K[2], M[1]): 123.23,
    (K[3], M[0]): 84.05, (K[3], M[1]): 111.50,
    
    (0, K[0]): 93.23, (0, K[1]): 70.72, (0, K[2]): 76.09, (0, K[3]): 102.92,
    (1, K[0]): 130.35, (1, K[1]):  112.76, (1, K[2]):  73.70, (1, K[3]):  102.62,
    (2, K[0]): 69.98, (2, K[1]):  52.39, (2, K[2]):  23.78, (2, K[3]): 45.68,
    (3, K[0]): 97.26, (3, K[1]):  89.65, (3, K[2]):  48.51, (3, K[3]): 41.06,
    (4, K[0]):  69.98, (4, K[1]):  73.70, (4, K[2]):  53.43, (4, K[3]): 24.57,
}
for j in M:
    for o in O:
        cost[j, o] = 0

In [185]:
# cost = {
#     (0, M[0]): 20879, (0, M[1]): 27031,
#     (1, M[0]): 22220, (1, M[1]): 25474,
#     (2, M[0]): 15574, (2, M[1]): 21866,
#     (3, M[0]): 14757, (3, M[1]): 17517,
#     (4, M[0]): 11935, (4, M[1]): 17718,
    
#     (K[0], M[0]): 6954.5681, (K[0], M[1]): 16155.4321,
#     (K[1], M[0]): 8523.9453, (K[1], M[1]): 14615.8621,
#     (K[2], M[0]): 10833.3003, (K[2], M[1]): 12306.5071,
#     (K[3], M[0]): 7266.6298, (K[3], M[1]): 12306.5071,
    
#     (0, K[0]): 8064.66, (0, K[1]): 5736.24, (0, K[2]): 6291.36, (0, K[3]): 9066.96,
#     (1, K[0]): 11904.24, (1, K[1]): 10084.68, (1, K[2]): 6044.64, (1, K[3]): 9036.12,
#     (2, K[0]): 5659.14, (2, K[1]): 3839.58, (2, K[2]): 880.482, (2, K[3]): 3145.68,
#     (3, K[0]): 8481, (3, K[1]): 7694.58, (3, K[2]): 3438.66, (3, K[3]): 2667.66,
#     (4, K[0]): 5659.14, (4, K[1]): 6044.64, (4, K[2]): 3947.52, (4, K[3]): 962.208,
# }
# for j in M:
#     for o in O:
#         cost[j, o] = 0


In [186]:
print(f'ori: {N}\n port: {M}\n trans: {K}')

ori: [0, 1, 2, 3, 4]
 port: [9, 10]
 trans: [5, 6, 7, 8]


In [187]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

print(oferta_total, demanda_total)

2749424 2749424


In [188]:
m = gp.Model("probrema")

In [189]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_port_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_tranship_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="tranship_{}_port_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [190]:
m.setObjective(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    sense=gp.GRB.MINIMIZE
)

In [191]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i]
        )

In [192]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [193]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_transbordo[k]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [194]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [195]:
m.Params.Method = -1

In [196]:
# Executa o modelo
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 40 columns and 156 nonzeros
Model fingerprint: 0x2ee0e6f4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 3e+06]
Presolve removed 10 rows and 2 columns
Presolve time: 0.00s
Presolved: 16 rows, 38 columns, 114 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0850983e+08   8.493989e+05   0.000000e+00      0s
      14    4.1032388e+08   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.103238758e+08


In [197]:
m.write("out.sol")

In [198]:
cap_inf = 408350021.71000004
cap_dem = 410323875.76000011